# Install packages

In [ ]:
!pip install aixplain
!pip install requests

# Set environment variables

In [ ]:
# environment variables
import os

# Fill in your aiXplain team API key
os.environ["TEAM_API_KEY"] = "<TEAM_API_KEY>"

# Imports

In [ ]:
from aixplain.factories import AgentFactory, ModelFactory, TeamAgentFactory

# LLMs & Onboarded Tools

In [ ]:
GPT_4O_MINI_ID = "669a63646eb56306647e1091"

web_search_tool = AgentFactory.create_model_tool(model="6736411cf127849667606689")

# Custom Tool: Publish to Threads

In [ ]:
def publish_to_threads(text: str):
    """
    Publishes a text post to Threads using the Threads API.

    Parameters:
    - text (str): The text content to post

    Returns:
    - str: The permalink URL of the published post

    Raises:
    - Exception: If any API calls fail
    """
    import requests
    import time

    # Fill in your Threads account information
    THREADS_USER_ID = "<THREADS_USER_ID>"
    ACCESS_TOKEN = "<ACCESS_TOKEN>"

    # Step 1: Create the media container
    create_url = f"https://graph.threads.net/v1.0/{THREADS_USER_ID}/threads"
    create_params = {
        "media_type": "TEXT",
        "text": text,
        "access_token": ACCESS_TOKEN
    }

    response = requests.post(create_url, params=create_params)
    if not response.ok:
        raise Exception(f"Failed to create media container: {response.text}")

    container_id = response.json()["id"]

    # Wait for server processing (recommended 30 seconds)
    time.sleep(30)

    # Step 2: Publish the container
    publish_url = f"https://graph.threads.net/v1.0/{THREADS_USER_ID}/threads_publish"
    publish_params = {
        "creation_id": container_id,
        "access_token": ACCESS_TOKEN
    }

    response = requests.post(publish_url, params=publish_params)
    if not response.ok:
        raise Exception(f"Failed to publish post: {response.text}")

    media_id = response.json()["id"]

    # Step 3: Retrieve the permalink
    get_url = f"https://graph.threads.net/v1.0/{media_id}"
    get_params = {
        "fields": "permalink",
        "access_token": ACCESS_TOKEN
    }

    response = requests.get(get_url, params=get_params)
    if not response.ok:
        raise Exception(f"Failed to retrieve post permalink: {response.text}")

    return response.json()["permalink"]

In [ ]:
publish_to_threads_model = ModelFactory.create_utility_model(name="Publish to Threads", code=publish_to_threads)
publish_to_threads_tool = AgentFactory.create_model_tool(model=publish_to_threads_model.id)

# Content Generator Agent

In [ ]:
CONTENT_GENERATOR_DESCRIPTION = "An agent that generates concise, appealing social media posts based on user input."

CONTENT_GENERATOR_INSTRUCTIONS = """
Generate an appealing text content for a social media post based on the user input.
You may search the web if you need more context or references.

Keep it concise to avoid boring viewers. You do not need to write full sentences.

CAUTIONS:
- The content must be less than 500 characters.
- Only one hashtag can be added in the post.
"""

content_generator_agent = AgentFactory.create(
    name="Content Generator",
    description=CONTENT_GENERATOR_DESCRIPTION,
    instructions=CONTENT_GENERATOR_INSTRUCTIONS,
    llm_id=GPT_4O_MINI_ID,
    tools=[web_search_tool],
)

# Content Reviewer Agent

In [ ]:
CONTENT_REVIEWER_DESCRIPTION = "An agent that reviews social media post content based on a user profile."

CONTENT_REVIEWER_INSTRUCTIONS = """
Review given text content for a social media post based on the given user profile.

If the content looks okay, just output "No feedbacks" without any comments.
If the content needs to be improved, give actionable feedback to modify the text.

CAUTION:
- Respect the given profile.
- Do not merely repeat the given content.
- Mention only relevant parts of the content that you want to give feedback on.
"""

content_reviewer_agent = AgentFactory.create(
    name="Content Reviewer",
    description=CONTENT_REVIEWER_DESCRIPTION,
    instructions=CONTENT_REVIEWER_INSTRUCTIONS,
    llm_id=GPT_4O_MINI_ID,
    tools=[],
)

# Content Editor Agent

In [ ]:
CONTENT_EDITOR_DESCRIPTION = "An agent that edits social media post content based on provided feedback."

CONTENT_EDITOR_INSTRUCTIONS = """
Modify given text content for a social media post based on the given user feedback.

Update only the parts of the given content that are mentioned in the feedback.

CAUTION:
- Use web search tool only when necessary.
"""

content_editor_agent = AgentFactory.create(
    name="Content Editor",
    description=CONTENT_EDITOR_DESCRIPTION,
    instructions=CONTENT_EDITOR_INSTRUCTIONS,
    llm_id=GPT_4O_MINI_ID,
    tools=[web_search_tool],
)

# Content Publisher Agent

In [ ]:
CONTENT_PUBLISHER_DESCRIPTION = "An agent that performs final review and publishes the content to Threads."

CONTENT_PUBLISHER_INSTRUCTIONS = """
Perform a final check on the generated content and publish it to the predefined Threads account.

Final checks include:
- Limit the content length to 500 characters.
- Remove any profanity words or offensive languages.

CAUTION:
- The user should have mentioned a positive feedback before publishing. Please check the chat history.
- Call the publishing tool only once.
"""

content_publisher_agent = AgentFactory.create(
    name="Content Publisher",
    description=CONTENT_PUBLISHER_DESCRIPTION,
    instructions=CONTENT_PUBLISHER_INSTRUCTIONS,
    llm_id=GPT_4O_MINI_ID,
    tools=[publish_to_threads_tool],
)

# Team Agent

In [ ]:
TEAM_DESCRIPTION = """
You are a social media expert who can create an appealing text content based on the user input and profile and publish it to a Threads account.

You might be asked to update the content through a conversation with the user.
If the content input is not given, refer to the chat history to fetch the latest generated content.

During the execution, please keep a high standard on the content quality; verify the generated content and improve incrementally, multiple times if needed.

Only when the user feedback is positive, e.g. "okay, good to go", publish it to Threads without further reviews.

FINAL OUTPUT FORMAT:
- When the content is generated/edited: content
- When the content is published: permalink of the content on Threads
- Do not include comments, explanations, section titles or any other information.

CAUTIONS:
* Do not merely perform all steps by calling all agents. Analyze the user input to decide which steps are relevant, and perform only those steps.
* If user feedback is not given, you must generate a content first; do not publish it.
* Respect what the user says.
* If the user feedback has too many critiques or encourages a fundamental change, consider completely regenerating the content.
* Do not perform publishing step more than once.
"""

team_agent = TeamAgentFactory.create(
    name="Threads Posting Agent",
    agents=[content_generator_agent, content_reviewer_agent, content_editor_agent, content_publisher_agent],
    description=TEAM_DESCRIPTION,
    use_mentalist_and_inspector=True,
)

# Prepare for Run

In [ ]:
# query format

QUERY = """
Profile:
{{profile}}

Content input:
{{content_input}}

User feedback:
{{user_feedback}}
"""

In [ ]:
# function for printing verbose output

def print_response(response):
    print(f"OUTPUT:\n{response.data['output']}")

    if response.data["intermediate_steps"][0]["agent"] == "mentalist":
        plan_str = response.data["intermediate_steps"][0]["output"]
        plan_list = [step.strip("{}") for step in plan_str.strip("[{}]").split("}, ")]
        print("\n\nORIGINAL PLAN:")
        print(*plan_list, sep='\n')

    print("\n\nINTERMEDIATE STEPS:")
    print(*response.data["intermediate_steps"], sep='\n')

    feedbacks = [
        step["output"] for step in response.data["intermediate_steps"]
        if step["agent"] == "Content Reviewer"
    ]
    print(f"\n\nREVIEW PROCESS: {len(feedbacks)} feedbacks")
    for i, feedback in enumerate(feedbacks):
        print(f"{i+1}: {feedback}")

    print(f"\nSession ID: {response.data['session_id']}")

# 1st Run

In [ ]:
profile = """
Professional business coaching consultant.
Be concise and friendly, but avoid flowery expressions.
Focus on actionable implementations and useful information.
"""

content_input = """
One most unknown yet clever productivity hack.
"""

user_feedback = "None"

response = team_agent.run(query=QUERY,
                          content={"profile": profile,
                                   "content_input": content_input,
                                   "user_feedback": user_feedback})

In [ ]:
print_response(response)

# 2nd Run: Edit

In [ ]:
user_feedback = """
Add more details on how to implement the hack. Add one or two emojis for visibility.
"""

# give generated content as the input in a new session
edit_response = team_agent.run(query=QUERY,
                               content={"profile": profile,
                                        "content_input": response.data["output"],
                                        "user_feedback": user_feedback})

In [ ]:
print_response(edit_response)

# 3rd Run: Publish

In [ ]:
content_input = "None"

user_feedback = """
Okay, please publish this.
"""

# give generated content as the input in a new session
publish_response = team_agent.run(query=QUERY,
                                  content={"profile": profile,
                                           "content_input": edit_response.data["output"],
                                           "user_feedback": user_feedback})

In [ ]:
print_response(publish_response)